# Dependencies installation and import 



In [ ]:
!pip install unidecode
!pip install pandas
!pip install re
!pip install pickle
!pip install transformers
!pip install numpy

In [ ]:
import pandas as pd
import re
import pickle 
import numpy as np
import os
from transformers import pipeline
import unidecode
from string import ascii_letters, punctuation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Prepare data

In [ ]:
wikihow = pd.read_csv(r'/content/drive/MyDrive/summarization/wikihowAll.csv')

In [ ]:
wikihow = wikihow.astype(str)
wikihow=wikihow[wikihow['text'].isnull()==False]
wikihow=wikihow[wikihow['headline'].isnull()==False]
wikihow=wikihow[wikihow['text']!='nan']
wikihow.drop_duplicates(subset=['text'],inplace=True)
wikihow['word_count'] = wikihow['text'].str.count(' ') + 1
wikihow['headline_count'] = wikihow['headline'].str.count(' ') + 1

In [ ]:
path = '/content/drive/MyDrive/summarization/wikihow_all.pkl'

In [ ]:
pd.to_pickle(wikihow, path)

In [ ]:
# wikihow_ = pd.read_pickle(path)

In [ ]:
contraction = { 
    "ain't": "is not", 
    "aren't": "are not", 
    "can't": "cannot", 
    "'cause": "because", 
    "could've": "could have", 
    "couldn't": "could not",
    "didn't": "did not", 
    "doesn't": "does not", 
    "don't": "do not", 
    "hadn't": "had not", 
    "hasn't": "has not", 
    "haven't": "have not",
    "haven t": "have not",
    "he'd": "he would", 
    "he'll": "he will", 
    "he's": "he is", 
    "how'd": "how did", 
    "how'd'y": "how do you", 
    "how'll": "how will", 
    "how's": "how is",
    "I'd": "I would", 
    "I'd've": "I would have", 
    "I'll": "I will", 
    "I'll've": "I will have", 
    "I'm": "I am", 
    "I've": "I have", 
    "i'd": "i would",
    "i'd've": "i would have", 
    "i'll": "i will", 
    "i'll've": "i will have", 
    "i'm": "i am", 
    "i've": "i have", 
    "isn't": "is not", 
    "it'd": "it would",
    "it'd've": "it would have", 
    "it'll": "it will", 
    "it'll've": "it will have", 
    "it's": "it is", 
    "let's": "let us", 
    "ma'am": "madam",
    "mayn't": "may not", 
    "might've": "might have", 
    "mightn't": "might not", 
    "mightn't've": "might not have", 
    "must've": "must have",
    "mustn't": "must not", 
    "mustn't've": "must not have", 
    "needn't": "need not", 
    "needn't've": "need not have", 
    "o'clock": "of the clock",
    "oughtn't": "ought not", 
    "oughtn't've": "ought not have", 
    "shan't": "shall not", 
    "sha'n't": "shall not", 
    "shan't've": "shall not have",
    "she'd": "she would", 
    "she'd've": "she would have", 
    "she'll": "she will", 
    "she'll've": "she will have", 
    "she's": "she is",
    "should've": "should have", 
    "shouldn't": "should not", 
    "shouldn't've": "should not have", 
    "so've": "so have", 
    "so's": "so as",
    "this's": "this is", 
    "that'd": "that would", 
    "that'd've": "that would have", 
    "that's": "that is", 
    "there'd": "there would",
    "there'd've": "there would have", 
    "there's": "there is", 
    "here's": "here is", 
    "they'd": "they would", 
    "they'd've": "they would have",
    "they'll": "they will", 
    "they'll've": "they will have", 
    "they're": "they are", 
    "they've": "they have", 
    "to've": "to have",
    "wasn't": "was not", 
    "we'd": "we would", 
    "we'd've": "we would have", 
    "we'll": "we will", 
    "we'll've": "we will have", 
    "we're": "we are",
    "we've": "we have", 
    "weren't": "were not", 
    "what'll": "what will", 
    "what'll've": "what will have", 
    "what're": "what are",
    "what's": "what is", 
    "what've": "what have", 
    "when's": "when is", 
    "when've": "when have", 
    "where'd": "where did", 
    "where's": "where is",
    "where've": "where have", 
    "who'll": "who will", 
    "who'll've": "who will have", 
    "who's": "who is", 
    "who've": "who have",
    "why's": "why is", 
    "why've": "why have", 
    "will've": "will have", 
    "won't": "will not", 
    "won't've": "will not have",
    "would've": "would have", 
    "wouldn't": "would not", 
    "wouldn't've": "would not have", 
    "y'all": "you all",
    "y'all'd": "you all would", 
    "y'all'd've": "you all would have", 
    "y'all're": "you all are", 
    "y'all've": "you all have",
    "you'd": "you would", 
    "you'd've": "you would have", 
    "you'll": "you will", 
    "you'll've": "you will have",
    "you're": "you are", 
    "you've": "you have"}

In [ ]:
def clean(text):
  
    text = text.lower()
    text = unidecode.unidecode(text)
    text = text.replace('&', ' and ')
    text = text.replace('@', ' at ')
    text = ' '.join([contraction[t] if t in contraction else t for t in text.split(" ")])
    
    punct = '|'.join('\\' + char for char in punctuation)
    punct = r'['+punct+']'
    text = re.sub(punct, ' ', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"'s\b","",text)
    text = re.sub(r"^\d+\s|\s\d+\s|\s\d+$", " ", text)

    text = text.strip(" ")
    text = re.sub('"','', text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(' +',' ', text).strip() 

    if len(text) != 0 and text[len(text)-1].isdigit():
        text = text[0:len(text)-1]
    return text



In [ ]:
wikihow["clean_title"] = wikihow['title'].map(lambda text: clean(text))

In [ ]:
wikihow["clean_text"] = wikihow['text'].map(lambda text: clean(text))

In [ ]:
wikihow["clean_headline"] = wikihow['headline'].map(lambda text: clean(text))

## Load WikiHow (test)


In [ ]:
# path = '/content/drive/MyDrive/summarization/wikihow_all_clean.pkl'
# pd.to_pickle(wikihow, path)

In [ ]:
# wikihow = pd.read_pickle(path)

# Summarize short articles

In [ ]:
shorty = wikihow["word_count"] < 800

short_articles = wikihow[shorty].copy()

In [ ]:
path = '/content/drive/MyDrive/summarization/short_articles_clean.pkl'
pd.to_pickle(short_articles, path)
short_articles

# Default Bart model

In [ ]:
def write_to(data, filename):
    filehandler = open(filename, 'wb') 
    pickle.dump(data, filehandler)

In [ ]:
bart_summarizer = pipeline(
    "summarization"
)

In [ ]:
short_articles['summary'] = ""

In [ ]:
filename = '/content/drive/MyDrive/summarization/summaries/short_articles.pkl'

for ind in short_articles['clean_text'].index:
  s = bart_summarizer(short_articles['clean_text'][ind], max_length=150)
  short_articles['summary'][ind] = s
  if ind > 0 and ind % 10 == 0:
    print('pickled')
    pd.to_pickle(short_articles, filename)

pd.to_pickle(short_articles, filename)

# Summarize shorter articles, T5 model



In [ ]:
shsh = wikihow["word_count"] < 300

shorter_articles = wikihow[shsh].copy()

In [ ]:
t5_summarizer = pipeline(
    "summarization",
    model="t5-base",
    tokenizer="t5-base",
    framework="tf"
)

In [ ]:
shorter_articles['summary'] = ""

In [ ]:
filename = '/content/drive/MyDrive/summarization/summaries/shorter_articles.pkl'

for ind in shorter_articles.index:
  text = shorter_articles['clean_text'][ind]  
  shorter_articles['summary'][ind]=t5_summarizer(text)
  
  if ind > 0 and ind % 100 == 0:
    pd.to_pickle(shorter_articles, filename)
    print('pickled')

pd.to_pickle(shorter_articles, filename)


# Evaluation


## Installation von nlg-eval, bleurt, functions

In [ ]:
import io

def write_list_to(filename, da_list):
  with open(filename, 'w') as f:
    for item in da_list:
      f.write("%s\n" % item)


def calculate_bleurt(filename):
  with io.open(filename,'r') as f:
    bl = f.readlines()

  b = [float(i) for i in bl]
  score = sum(b)/len(b)
  print('score for ' + filename)
  print(score)


In [ ]:
!pip3 install git+https://github.com/Maluuba/nlg-eval.git@master


In [ ]:
!nlg-eval --setup

In [ ]:
!git clone https://github.com/google-research/bleurt.git

In [ ]:
cd bleurt/

In [ ]:
!pip3 install .

In [ ]:
!python -m unittest bleurt.score_test
!python -m unittest bleurt.score_not_eager_test
!python -m unittest bleurt.finetune_test

In [ ]:
!wget https://storage.googleapis.com/bleurt-oss/bleurt-large-512.zip .

In [ ]:
!unzip bleurt-large-512.zip

## Short Articles summary evaluation




In [ ]:
to_eval_s = short_articles[0:10]

In [ ]:
to_eval_hyp_s = [x[0].get('summary_text') for x in to_eval['summary']]
to_eval_ref_s = to_eval['clean_headline']

In [ ]:
h_filename_s = '/content/drive/MyDrive/summarization/evaluation/short_articles_hypothesis.txt'
r_filename_s = '/content/drive/MyDrive/summarization/evaluation/short_articles_reference.txt'

In [ ]:
write_list_to(h_filename_s, to_eval_hyp_s)
write_list_to(r_filename_s, to_eval_ref_s)

In [ ]:
!nlg-eval --hypothesis=/content/drive/MyDrive/summarization/evaluation/short_articles_hypothesis.txt --references=/content/drive/MyDrive/summarization/evaluation/short_articles_reference.txt --no-skipthoughts --no-glove 

In [ ]:
!python -m bleurt.score \
  -candidate_file=/content/drive/MyDrive/summarization/evaluation/short_articles_hypothesis.txt \
  -reference_file=/content/drive/MyDrive/summarization/evaluation/short_articles_reference.txt \
  -bleurt_checkpoint=/content/bleurt/bleurt-large-512\
  -scores_file=bleurt_scores_short_articles_summary.txt

In [ ]:
calculate_bleurt('bleurt_scores_short_articles_summary.txt')

## Shorter Articles summary evaluation

In [ ]:
to_eval = shorter_articles[0:20]

In [ ]:
to_eval_hyp = [x[0].get('summary_text') for x in to_eval['summary']]
to_eval_ref = to_eval['clean_headline']

In [ ]:
h_filename = '/content/drive/MyDrive/summarization/evaluation/shorter_articles_hypothesis.txt'
r_filename = '/content/drive/MyDrive/summarization/evaluation/shorter_articles_reference.txt'

In [ ]:
write_list_to(h_filename, to_eval_hyp)
write_list_to(r_filename, to_eval_ref)

In [ ]:
!nlg-eval --hypothesis=/content/drive/MyDrive/summarization/evaluation/shorter_articles_hypothesis.txt --references=/content/drive/MyDrive/summarization/evaluation/shorter_articles_reference.txt --no-skipthoughts --no-glove 

In [ ]:
!python -m bleurt.score \
  -candidate_file=/content/drive/MyDrive/summarization/evaluation/shorter_articles_hypothesis.txt \
  -reference_file=/content/drive/MyDrive/summarization/evaluation/shorter_articles_reference.txt \
  -bleurt_checkpoint=/content/bleurt/bleurt-large-512\
  -scores_file=bleurt_scores_shorter_articles_summary.txt

In [ ]:
calculate_bleurt('bleurt_scores_shorter_articles_summary.txt')

# Manual evaluation TODO

In [ ]:
with open("/content/guardian.txt", "r", encoding="utf-8") as f:
        guardian = " ".join(f.readlines())
        
t5_summary = t5_summarizer(guardian)



In [ ]:
bart_summary = bart_summarizer(guardian)